In [ ]:
from __future__ import print_function, division

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize

from astropy.modeling import models, fitting, \
                             FittableModel, Parameter

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs
    
from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status

In [ ]:
class Momentum_Surface_Point(object):
    def __init__(self, simulation, status):
        self.mass        = simulation.cluster_mass
        self.momentum    = simulation.momentum
        self.density     = simulation.background_density
        self.metallicity = simulation.metallicity
        self.id          = simulation.id
        self.status      = status
        self.num_SNe     = simulation.num_SNe

In [ ]:
points = []

for simulation in session.query(Simulation):
    status = session.query(Simulation_Status).get(simulation.id)
    if status is None:
        continue
    if status.status in ["Error", "Unphysical"]:
        simple_status = "bad"
    else:
        simple_status = "good"
    point = Momentum_Surface_Point(simulation, simple_status)
    points.append(point)

In [ ]:
metallicities = np.array([point.metallicity for point in points])
densities =     np.array([point.density     for point in points])
masses =        np.array([point.mass        for point in points])
momenta =       np.array([point.momentum    for point in points])
statuses =      np.array([point.status      for point in points])
usable =        np.full_like(masses, False, dtype=bool)
excluded =      ((densities > m_proton) & (masses < 10**3.1 * M_solar))
num_SNe  =      np.array([point.num_SNe     for point in points])


momenta = momenta / masses / 1e5 # [km / s]
masses = masses / M_solar # [M_solar]


metallicities_1D = np.sort(list(set(metallicities)))
densities_1D     = np.sort(list(set(densities)))
masses_1D        = np.sort(list(set(masses)))

# if one initialization is bad, all initializations are considered bad
# (we can't average if at least one is unphysical)
for metallicity in metallicities_1D:
    for density in densities_1D:
        for mass in masses_1D:
            mask = np.isclose(metallicities, metallicity, atol=0) \
                 & np.isclose(densities,     density,     atol=0) \
                 & np.isclose(masses,        mass,        atol=0)
            usable[mask] = np.all(statuses[mask] == "good")

metallicities_3D, densities_3D, masses_3D = np.meshgrid(metallicities_1D, densities_1D, masses_1D, 
                                                       indexing="ij") 

momenta_3D = np.empty_like(masses_3D)

for i, metallicity in enumerate(metallicities_1D):
    for j, density in enumerate(densities_1D):
        for k, mass in enumerate(masses_1D):

            good_metallicities = (metallicities == metallicity)
            good_densities     = (densities == density)
            good_masses        = (masses == mass)
            good = good_metallicities & good_densities & good_masses & usable
            
            momenta_3D[i,j,k] = np.average(momenta[good])


momenta_3D[momenta_3D == 0] = 1 # in case we take the log...



In [ ]:
class Momentum_Model(object):
    
    def __init__(self, 
                 norm_low,            norm_high,
                 eta_metallicity_low, eta_metallicity_high, 
                 eta_density_low,     eta_density_high, 
                 eta_N_SNe_low,       eta_N_SNe_high):
        
        params_0 = (norm_low,            norm_high,
                    eta_metallicity_low, eta_metallicity_high,
                    eta_density_low,     eta_density_high,
                    eta_N_SNe_low,       eta_N_SNe_high)
        
        self.params_0 = params_0
        self.params   = params_0
    
    def __call__(self, metallicity, density, N_SNe):
        
        norm_low,            norm_high, \
        eta_metallicity_low, eta_metallicity_high, \
        eta_density_low,     eta_density_high, \
        eta_N_SNe_low,       eta_N_SNe_high = self.params
        
        x = (metallicity, density, N_SNe)
        
        return self.evaluate(x,
                             np.log10(norm_low),  np.log10(norm_high),
                             eta_metallicity_low, eta_metallicity_high,
                             eta_density_low,     eta_density_high,
                             eta_N_SNe_low,       eta_N_SNe_high)
        
    def __str__(self):
        s  = "p/(100 * M_sun * N_SNe) ~ min("
        s +=       "{0:.2e}"
        s +=       " * (Z / Z_sun)**{1:.2f}"
        s +=       " * (rho / m_p)**{2:.2f}"
        s +=       " * (N_SNe/1)**{3:.2f}"
        s += " , "
        s +=       "{4:.2e}"
        s +=       " * (Z / Z_sun)**{5:.2f}"
        s +=       " * (rho / m_p)**{6:.2f}"
        s +=       " * (N_SNe/1000)**{7:.2f}"
        s += ")"
        s = s.format(self.params[0], self.params[2], self.params[4], self.params[6],
                     self.params[1], self.params[3], self.params[5], self.params[7])
        
        return s
    def fit(self, x, y, fixed=None, **kwargs):
        """Fits the model using its initial self.params_0;
        Stores result as self.params
        

        Parameters
        --------
            x : locations of initial N data points; array: N x 3
                - metallicity 
                - density [g cm**-3]
                - number of SNe 
                - momentum efficiency [km / s]
            y : momentum efficiency [g cm**-3] at those N data points; array: Nx1
            fixed : mask of values to be held fixed; None or np.ndarray (dtype=bool)
                - fixed values set to True
                - free  values set to False
                - None for all values free
            **kwargs : to be passed to optimize.curve_fit; dict
                - does NOT get passed to the fitted function


        Returns
        -------
        popt : best-fit parameters; array: 8x1
        pcov : estimated parameter covariance; array: n_free x n_free
            - where: n_free = sum(~fixed)


        Side effects
        ------------
        Overwrites self.params with latest fit results
        
        Notes
        -----
        Fits self.evaluate
        
        covariance for normalizations is covariance in the *log*
        
        Doesn't return pcov entries corresponding to fixed values.
        This might cause errors if you forget to check if a parameter was fixed.
        In the future this should be fixed.
        """
        p0 = np.array(self.params_0)
        p0[0:2] = np.log10(p0[0:2])
        
        if fixed is None:
            fixed = np.full_like(p0, False, dtype=bool)
        elif isinstance(fixed, np.ndarray):
            if not fixed.dtype == bool:
                raise TypeError("fixed must be an np.ndarray of bools")
        else:
            raise TypeError("fixed must be type None or np.ndarray")
            
        def _f(x, *args):
            p0[~fixed] = args
            return self.evaluate(x, *p0)
        
        popt, pcov = optimize.curve_fit(_f, x, y, p0=p0[~fixed],
                                        xtol=.01,
                                        **kwargs)
        p0[~fixed] = popt
        popt = p0
        popt[0:2] = 10**popt[0:2] # we fit the log of the normalization
        self.params = tuple(popt)
        return popt, pcov

    
    @staticmethod
    def evaluate(x,
                log_norm_low,        log_norm_high,
                eta_metallicity_low, eta_metallicity_high,
                eta_density_low,     eta_density_high,
                eta_N_SNe_low,       eta_N_SNe_high):
        
        norm_low  = 10**log_norm_low
        norm_high = 10**log_norm_high
        
        metallicity, density, N_SNe = x
        
        # Normalize p_M_low at M=10**2 M_solar
        p_M_low  = norm_low  \
                    * (metallicity / metallicity_solar)**eta_metallicity_low \
                    * (density / m_proton)**eta_density_low \
                    * (N_SNe / (1))**eta_N_SNe_low
        
        # Normalize p_M_low at M=10**5 M_solar
        p_M_high = norm_high \
                    * (metallicity / metallicity_solar)**eta_metallicity_high \
                    * (density / m_proton)**eta_density_high \
                    * (N_SNe / (1000))**eta_N_SNe_high
        
        p_M = (p_M_low * p_M_high) / (p_M_low + p_M_high)
            
        return p_M

        

## Example: visualize surface (fixed metallicity)

In [ ]:
%matplotlib notebook

metallicity_index = np.argmax(metallicities_1D==metallicity_solar)

from mpl_toolkits.mplot3d import axes3d
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

with sns.plotting_context("poster"):
    surf = ax.plot_wireframe(np.log10(masses_3D[   metallicity_index,:,:]),
                             np.log10(densities_3D[metallicity_index,:,:]),
                             momenta_3D[           metallicity_index,:,:],
                             rstride=1, cstride=1, linewidth=1)

    plt.xlabel("log Mass")
    plt.ylabel("log density")
    plt.show()

## Example: visualize surface (fixed density)

In [ ]:
%matplotlib notebook

density_index = np.argmax(densities_1D==1.33 * m_proton)

from mpl_toolkits.mplot3d import axes3d
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

with sns.plotting_context("poster"):
    surf = ax.plot_wireframe(np.log10(masses_3D[       :, density_index, :]),
                             np.log10(metallicities_3D[:, density_index, :]),
                             momenta_3D[               :, density_index, :],
                             rstride=1, cstride=1, linewidth=1)

    plt.xlabel("log Mass")
    plt.ylabel("log Z / Z_sun")
    plt.show()

## Test: call the model 

In [ ]:
%matplotlib inline

metallicity_index = np.argmax(metallicities_1D==metallicity_solar)
density_index = 2
metallicity = metallicities_1D[metallicity_index]
density     = densities_1D[density_index]
mask = np.isclose(densities, density, atol=0) & np.isclose(metallicities, metallicity, atol=0) & usable & (momenta>0)

tmp_model = Momentum_Model(1e3, 1e4, 
                           0, 0, 
                           0, 0, 
                           2, -.2)

print(tmp_model(metallicity, density, masses[mask]))

with sns.plotting_context("poster", font_scale=2):

    plt.plot(num_SNe[mask],
             momenta[mask],
             marker= "o", linestyle="",
             label="data")
    
    
    x_fit = np.logspace(0,3.5, num=100)
    y_fit = tmp_model(metallicity, density, x_fit)
    plt.plot(x_fit, y_fit,
            label="chi-by-eye")
    
    plt.xscale("log")
    plt.xlabel("N_SNe")
    plt.ylabel("Momentum / (100 M$_\odot$ * N$_\mathrm{SNe}$) [km / s]")
    plt.legend(loc="best")

## Test: fit the model (using 1D slice -- fixed density, metallicity)

In [ ]:
%matplotlib inline

metallicity_index = np.argmax(metallicities_1D==metallicity_solar)
density_index = 2
metallicity = metallicities_1D[metallicity_index]
density     = densities_1D[density_index]
mask = np.isclose(densities, density, atol=0) & np.isclose(metallicities, metallicity, atol=0) & usable & (momenta>0)

tmp_model = Momentum_Model(1e3, 1e4, 
                           0, 0, 
                           0, 0, 
                           2, -.12)
fixed = np.array([False, False, 
                  True, True, 
                  True, True, 
                  False, True])


y_init = tmp_model(metallicity, density, num_SNe[mask])
print("y_init: ", y_init)

x = (metallicities[mask],
     densities[mask],
     num_SNe[mask])
y = momenta[mask]
popt, pcov = tmp_model.fit(x, y, fixed=fixed)

print("params_0: ", tmp_model.params_0)
print("params:   ", tmp_model.params)



with sns.plotting_context("poster", font_scale=2):
    plt.plot(num_SNe[mask], momenta[mask],
             marker= "o", linestyle="",
             label="data")
    
    x_fit = np.logspace(0,3.5, num=100)
    y_fit = tmp_model(metallicity, density, x_fit)
    plt.plot(x_fit, y_fit,
             label="fit")
    plt.xscale("log")
    plt.legend(loc="best")
    plt.xlabel("N_SNe ")
    plt.ylabel("Momentum / (100 * M$_\odot$ * N$_\mathrm{SNe}$) [km / s]")

## Test: fit the model (using 2D slice of solar metallicity)

In [ ]:
%matplotlib inline

metallicity_index = np.argmax(metallicities_1D==metallicity_solar)
density_index = 2

model_overall = Momentum_Model(1e3, 1e4,
                               0, 0, 
                               .25, .2,
                               2, -.12)
fixed = np.array([False, False, 
                  False, False, 
                  False, False, 
                  False, True])

mask = usable & (momenta>0)
xs = (metallicities[mask],
      densities[mask],
      num_SNe[mask])
ys = momenta[mask]


popt, pcov = model_overall.fit(xs, ys, fixed=fixed)
print("params_0: ", model_overall.params_0)
print("params:   ", model_overall.params)
print()
print("overall model: ", str(model_overall))

models = []

with sns.plotting_context("poster", font_scale=2):
    for density_index in range(len(densities_1D)):
        metallicity = metallicities_1D[metallicity_index]
        density     = densities_1D[density_index]
        mask = np.isclose(densities, density, atol=0) \
             & np.isclose(metallicities, metallicity, atol=0) \
             & usable & (momenta>0) & ~excluded
        
        model_tmp = Momentum_Model(1e3, 1e4, 
                                   0, 0, 
                                   0, 0,
                                   2, -.12)
        fixed = np.array([False, False, 
                          True, True, 
                          True, True, 
                          False, True])
        
        x = [metallicities[mask],
             densities[mask],
             num_SNe[mask]]
        y = momenta[mask]
        
        fitted = False
        if len(x[0]) >= (~fixed).sum():
            model_tmp.fit(x,y, fixed=fixed)
            fitted = True
        models.append(model_tmp)
        fitted = False
        
        x_fit = np.logspace(0,3.25, num=100)
        y_fit = model_tmp(metallicity, density, x_fit)
        y_fit_overall = model_overall(metallicity, density, x_fit)
        
        print()
        print("=================")
        print("metallicity: ", metallicity)
        print("density:     ", density)
        print(str(model_tmp))
        plt.figure()
        plt.title("density = {0:e} g cm^-3".format(density))


        print("number plotted: ", sum(mask))
        plt.plot(num_SNe[mask], momenta[mask],
                 marker= "o", linestyle="",
                 label="data")
        if fitted:
            plt.plot(x_fit, y_fit,  label="fit", color="g")
        plt.plot(x_fit, y_fit_overall, label="overall fit", linestyle="--", color='r')
        plt.xscale("log")
        plt.legend(loc="best")
        plt.xlabel("N_SNe ")
        plt.ylabel("momentum / (100 * M$_\odot$ * N$_\mathrm{SNe}$) [km / s]")
        
        plt.xlim(np.min(x_fit)/3 , np.max(x_fit)*3)
        
        if density_index == 3:
            plt.xlim(num_SNe[mask].min()/3, num_SNe[mask].max()*3)
            plots_dir = "plots_for_paper"
            plt.tight_layout()
            plot_filename = os.path.join(plots_dir, "momentum-fit-sample")
            plt.savefig(plot_filename + ".eps")
            plt.savefig(plot_filename + ".pdf")
            plt.savefig(plot_filename + ".png")
        
        
        plt.show()
        
        

        

## Test: fit the model (using 2D slice -- fixed density)

In [ ]:
%matplotlib inline

density_index = np.argmax(np.isclose(densities_1D, 1.33e-1 * m_proton, atol=0, rtol=1e-4))

model_overall = Momentum_Model(1e3, 1e4,
                               0, 0, 
                               .25, .2,
                               2, -.12)
fixed = np.array([False, False, 
                  False, False, 
                  False, False, 
                  False, False])


mask = usable & (momenta>0)
xs = (metallicities[mask], densities[mask], num_SNe[mask])
ys = momenta[mask]

popt, pcov = model_overall.fit(xs, ys, fixed=fixed)
print("params_0: ", model_overall.params_0)
print("params:   ", model_overall.params)
print()
print("overall model: ", str(model_overall))

models = []

with sns.plotting_context("poster", font_scale=2):
    for metallicity_index in range(len(metallicities_1D)):
        metallicity = metallicities_1D[metallicity_index]
        density     = densities_1D[density_index]
        mask = np.isclose(densities, density, atol=0) \
             & np.isclose(metallicities, metallicity, atol=0) \
             & usable & (momenta>0) & ~excluded
        
        model_tmp = Momentum_Model(1e3, 1e4, 
                                   0, 0, 
                                   0, 0,
                                   2, -.12)
        fixed = np.array([False, False, 
                          True, True, 
                          True, True, 
                          False, True])
        
        x = (metallicities[mask], densities[mask], num_SNe[mask])
        y = momenta[mask]


        fitted = False
        if len(x[0]) >= (~fixed).sum():
            fitted = True
            model_tmp.fit(x,y, fixed=fixed)
        models.append(model_tmp)
        fitted = False
        
        x_fit = np.logspace(0, 3.25, num=100)
        y_fit = model_tmp(metallicity, density, x_fit)
        y_fit_overall = model_overall(metallicity, density, x_fit)
                                      
        print()
        print("=================")
        print("metallicity: ", metallicity)
        print("density:     ", density)
        print(str(model_tmp))
        plt.figure()
        plt.title("log Z / Z_solar = {0:.1f}".format(np.log10(metallicity / metallicity_solar)))

     
        print("number plotted: ", sum(mask))
        plt.plot(num_SNe[mask], momenta[mask],
                 marker= "o", linestyle="",
                 label="data")
        
        plt.xlim(np.min(x_fit)/3 , np.max(x_fit)*3)
        
        if fitted:
            plt.plot(x_fit, y_fit,  label="fit", color="g")
        plt.plot(x_fit, y_fit_overall, 
                 label="overall fit", linestyle="--", color="r")
        plt.xscale("log")
        plt.legend(loc="best", frameon=True, shadow=True)
        plt.xlabel("N_SNe ")
        plt.ylabel("momentum / (100 * M$_\odot$ * N$_\mathrm{SNe}$) [km / s]")
        plt.show()